In [ ]:
!pip install redis
!pip install pip install nemoguardrails==0.8.0

In [ ]:
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo
  
streaming: True
  
"""


In [ ]:
rag_colang_content = """
# Defind User Greeting Block
define user greeting
    "Hey There!"
    "How are you?"
    "What's Up?"

define bot greeting
    "Hello, How can I assist you?"

define flow greeting
    user greeting
    bot greeting

"""


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "provide api key"

In [ ]:
from nemoguardrails import RailsConfig, LLMRails

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
config = RailsConfig.from_content(rag_colang_content, yaml_content)
app = LLMRails(config,llm=llm)

In [ ]:
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.memory import ConversationBufferWindowMemory

chat_memory = RedisChatMessageHistory(
            url='redis://localhost:6379',
            # Your Redis database URL
            ttl=600,  # Time to live for the messages in the database
            session_id="test_session",
            key_prefix='chat_history'
        )
memory = ConversationBufferWindowMemory(memory_key="chat_history",
                                                     chat_memory=chat_memory, k=4,human_prefix='user',ai_prefix='assistant')

In [ ]:
from typing import List
from langchain_core.messages import HumanMessage
def getMessageObj() -> List[dict]:
    messages = []
    for message in memory.buffer_as_messages:
        content = message.content
        role = 'assistant'
        if type(message) is HumanMessage:
            role = 'user'
        messages.append({"role": role, "content": content})
    return messages

In [ ]:
input="where is my order"
messages = []
messages.append({"role": "context", "content": {'testIntentKey': "abc"}})
print(f"history = {getMessageObj()}")
messages.extend(getMessageObj())
messages.append({"role": "user", "content": input })
output = await app.generate_async(messages=messages)
print(f"the content = {output}")
memory.save_context({"input":input},{"output":output['content']})